
## Wine reviews analysis using PySpark NLP

**Dataset:**
https://www.kaggle.com/zynicide/wine-reviews


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
spark = SparkSession.builder.appName('WineReviewsML').getOrCreate()

conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

spark.sparkContext.getConf().getAll()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/29 14:51:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.app.startTime', '1706561477715'),
 ('spark.executor.id', 'driver'),
 ('spark.executor.memory', '5g'),
 ('spark.executor.cores', '4'),
 ('spark.app.id', 'local-1706561478372'),
 ('spark.cores.max', '4'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5

## Read Data

In [3]:
df = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv("/Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv",inferSchema=True, header=True )

In [4]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [5]:
df2 = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv("/Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data-130k-v2.csv",inferSchema=True, header=True )

In [6]:
df2.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- description: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- points: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- province: string (nullable = true)
 |-- region_1: string (nullable = true)
 |-- region_2: string (nullable = true)
 |-- taster_name: string (nullable = true)
 |-- taster_twitter_handle: string (nullable = true)
 |-- title: string (nullable = true)
 |-- variety: string (nullable = true)
 |-- winery: string (nullable = true)



In [7]:
df = df.union(df2.drop("taster_name", "taster_twitter_handle", "title"))

## Data Exploration

In [8]:
df.count()

280901

In [9]:
df.show(5)

+---+-------+--------------------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+
|_c0|country|         description|         designation|points|price|      province|         region_1|         region_2|           variety|              winery|
+---+-------+--------------------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|    California|      Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|Northern Spain|             Toro|             NULL|     Tinta de Toro|Bodega Carmen Rod...|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|    California|   Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|  3|     US|This spent 20 mon...|      

24/01/29 14:53:18 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


In [10]:
df.dropna().count()

24/01/29 14:53:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv
24/01/29 14:53:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data-130k-v2.csv


73325

##  Feature Engineering

**QuantileDiscretizer** takes a column with continuous features and outputs a column with binned categorical features. The number of bins can be set using the numBuckets parameter. It is possible that the number of buckets used will be smaller than this value, for example, if there are too few distinct values of the input to create enough distinct quantiles. Since 2.3.0, QuantileDiscretizer can map multiple columns at once by setting the inputCols parameter. If both of the inputCol and inputCols parameters are set, an Exception will be thrown. To specify the number of buckets for each column, the numBucketsArray parameter can be set, or if the number of buckets should be the same across columns, numBuckets can be set as a convenience. 

In [11]:
from pyspark.ml.feature import QuantileDiscretizer

discretizer = QuantileDiscretizer(numBuckets=3, inputCol="points", outputCol="points_category")
df = discretizer.fit(df).transform(df)
df.show(3)

+---+-------+--------------------+--------------------+------+-----+--------------+--------------+--------+------------------+--------------------+---------------+
|_c0|country|         description|         designation|points|price|      province|      region_1|region_2|           variety|              winery|points_category|
+---+-------+--------------------+--------------------+------+-----+--------------+--------------+--------+------------------+--------------------+---------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|    California|   Napa Valley|    Napa|Cabernet Sauvignon|               Heitz|            2.0|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|Northern Spain|          Toro|    NULL|     Tinta de Toro|Bodega Carmen Rod...|            2.0|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|    California|Knights Valley|  Sonoma|   Sauvignon Blanc|            Macauley|            2.0|
+---+-------+---

24/01/29 14:53:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


## Natural Language Processing

In [12]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

tokenizer = Tokenizer(inputCol="description", outputCol="words")
df = tokenizer.transform(df)

#dropping the redundant column
df= df.drop("description")
df.show(5)

+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+
|_c0|country|         designation|points|price|      province|         region_1|         region_2|           variety|              winery|points_category|               words|
+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+
|  0|     US|   Martha's Vineyard|    96|235.0|    California|      Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|            2.0|[this, tremendous...|
|  1|  Spain|Carodorum Selecci...|    96|110.0|Northern Spain|             Toro|             NULL|     Tinta de Toro|Bodega Carmen Rod...|            2.0|[ripe, aromas, of...|
|  2|     US|Special Selected ...|    96| 90.0|    California|   Knights Valley|           Sonoma|   Sauvignon Blanc|   

24/01/29 14:54:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


In [13]:
from pyspark.ml.feature import StopWordsRemover

#remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
df = remover.transform(df)

#drop the redundant column
df= df.drop("words")
df.show(5,truncate = False)

+---+-------+------------------------------------+------+-----+--------------+-----------------+-----------------+------------------+-----------------------+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0|country|designation                         |points|price|province      |region_1         |region_2         |variety           |winery                 |points_category|filtered                                                                                                                                                                                                                                                                                                 

24/01/29 14:54:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


In [14]:
#Mapping a sequence of terms to their term frequencies using the hashing trick. 
#alternatively, CountVectorizer can also be used to get term frequency vectors

hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(df)

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
nlpdf = idfModel.transform(featurizedData)
nlpdf.select("points_category", "features").show(5, truncate=False)

+---------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|points_category|features                                                                                                                                                                                                                                                                                                                                                                                                                             |
+---------------+-------------------------------------------------------------------------------------------------------

In [15]:
nlpdf.show(5)

+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+--------------------+--------------------+
|_c0|country|         designation|points|price|      province|         region_1|         region_2|           variety|              winery|points_category|            filtered|         rawFeatures|            features|
+---+-------+--------------------+------+-----+--------------+-----------------+-----------------+------------------+--------------------+---------------+--------------------+--------------------+--------------------+
|  0|     US|   Martha's Vineyard|    96|235.0|    California|      Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|            2.0|[tremendous, 100%...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
|  1|  Spain|Carodorum Selecci...|    96|110.0|Northern Spain|             Toro|             NULL|     Tinta de Toro|Bodega Carm

24/01/29 14:54:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


In [16]:
train_df, test_df = nlpdf.randomSplit([0.8, 0.2])

train_df.show(1)

24/01/29 14:54:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+--------------------+
|_c0|country|      designation|points|price|  province|   region_1|region_2|           variety|winery|points_category|            filtered|         rawFeatures|            features|
+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+--------------------+
|  0|     US|Martha's Vineyard|    96|235.0|California|Napa Valley|    Napa|Cabernet Sauvignon| Heitz|            2.0|[tremendous, 100%...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+--------------------+
only showing top 1 row



### Logistic Regression Model

In [17]:
from pyspark.ml.classification import LogisticRegression

lgr = LogisticRegression(maxIter=10, featuresCol = 'features', labelCol='points_category')

lgrm = lgr.fit(train_df)

predictions = lgrm.transform(test_df)

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=
24/01/29 14:54:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


CodeCache: size=131072Kb used=39319Kb max_used=39500Kb free=91752Kb
 bounds [0x000000010b1e0000, 0x000000010d8c0000, 0x00000001131e0000]
 total_blobs=14085 nmethods=13056 adapters=940
 compilation: disabled (not enough contiguous free space left)


24/01/29 14:54:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data-130k-v2.csv
24/01/29 14:55:00 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/01/29 14:55:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-da

In [18]:
predictions.show(3)

24/01/29 14:55:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


+---+-------+------------------+------+-----+------------------+------------------+-----------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|_c0|country|       designation|points|price|          province|          region_1|         region_2|   variety|              winery|points_category|            filtered|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+---+-------+------------------+------+-----+------------------+------------------+-----------------+----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  8|     US|            Silice|    95| 65.0|            Oregon|Chehalem Mountains|Willamette Valley|Pinot Noir|           Bergström|            2.0|[re-named, vineya...|(20,[0,3,4,5,7,8,...|(20,[0,3,4,

### Evaluation

In [19]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="points_category", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

24/01/29 14:55:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv
24/01/29 14:55:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data-130k-v2.csv
24/01/29 14:55:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Heade

0.5244819919964429


24/01/29 14:55:49 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data-130k-v2.csv


0.5208610877302592


### Word2Vec

The Word2VecModel transforms each document into a vector using the average of all words in the document ; this vector can then be used as features for prediction, document similarity calculations, etc.

In [20]:

from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=50, minCount=0, inputCol="filtered", outputCol="wordVectors")
w2VM = word2Vec.fit(df)
nlpdf = w2VM.transform(df)

In [21]:
nlpdf.select("points_category", "wordVectors").show(2, truncate=False)

+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:

splits = nlpdf.randomSplit([0.8, 0.2])

train_df = splits[0]
test_df = splits[1]

train_df.show(1)

24/01/29 14:57:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv


+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+
|_c0|country|      designation|points|price|  province|   region_1|region_2|           variety|winery|points_category|            filtered|         wordVectors|
+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+
|  0|     US|Martha's Vineyard|    96|235.0|California|Napa Valley|    Napa|Cabernet Sauvignon| Heitz|            2.0|[tremendous, 100%...|[0.10064252620698...|
+---+-------+-----------------+------+-----+----------+-----------+--------+------------------+------+---------------+--------------------+--------------------+
only showing top 1 row



In [23]:
lgr = LogisticRegression(maxIter=20, featuresCol = 'wordVectors', labelCol='points_category')

lgrm = lgr.fit(train_df)

predictions = lgrm.transform(test_df)

24/01/29 14:57:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv
24/01/29 14:57:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data-130k-v2.csv
24/01/29 14:57:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Heade

## Evaluation

In [24]:
evaluator = MulticlassClassificationEvaluator(labelCol="points_category", predictionCol="prediction")

print(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"}))
print(evaluator.evaluate(predictions, {evaluator.metricName: "f1"}))

24/01/29 14:58:03 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data_first150k.csv
24/01/29 14:58:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data-130k-v2.csv
24/01/29 14:58:05 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Heade

0.6339731696874554


24/01/29 14:58:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , country, description, designation, points, price, province, region_1, region_2, variety, winery
 Schema: _c0, country, description, designation, points, price, province, region_1, region_2, variety, winery
Expected: _c0 but found: 
CSV file: file:///Users/saiomkarkandukuri/Desktop/github/pyspark-projects/wine-reviews-appliedml/data/winemag-data-130k-v2.csv


0.6331508530523093


24/01/29 17:00:38 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1890856 ms exceeds timeout 120000 ms
24/01/29 17:00:38 WARN SparkContext: Killing executors is not supported by current scheduler.
24/01/29 17:00:41 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$